# Processing guidelies: 
1. Downsample to 512
2. reference to Cz
3. filter ([.1,120] and line noise)
4. remove bad channels (not done here)
5. Epoch data 
6. inspect epochs (not done here)
7. save as mne .fif and as array 


In [1]:
#!pip install git+https://github.com/DiGyt/asrpy.git -q

In [1]:
from ast import literal_eval
from asrpy import ASR
import gc
import json
import matplotlib
#matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import mne
import numpy as np
import os
import pandas as pd
import pyxdf
import seaborn as sns

In [2]:
#set up the datasourcs and folder structure
datasrc = "/net/store/nbp/projects/GTI_decoding/data/pilot"
alltrials_file = "02_general_exp_data/allTrialsData.csv"
experimentdetails_file = "02_general_exp_data/ExperimentDetails.csv"
montagefile = "../misc/ANT_EEG_channel_mapping.json"

#folder structure: data/pilot/01_subjects_data/P001/eeg/preprocessed/04_PCA
subj_dir = "01_subjects_data"
dir_structure = {
    'events': "eeg/preprocessed/00_events_added",
    'filtered': "eeg/preprocessed/01_filtered",
    'epoched': "eeg/preprocessed/02_epoched",
    'cleaned': "eeg/preprocessed/03_cleaned",
    'pca': "eeg/preprocessed/04_PCA",
    'rawepochs': "eeg/preprocessed/99_epochs_raw"
    }
    

In [5]:
trial_types = ["/".join([x,y,z]) for x in ['lift','use'] for y in ['fam','unfam'] for z in ['left','right']]
trial_event_map = {t:i+1 for i,t in enumerate(trial_types)}

In [6]:
trial_event_map

{'lift/fam/left': 1,
 'lift/fam/right': 2,
 'lift/unfam/left': 3,
 'lift/unfam/right': 4,
 'use/fam/left': 5,
 'use/fam/right': 6,
 'use/unfam/left': 7,
 'use/unfam/right': 8}

In [7]:
expt_df = pd.read_csv(os.path.join(datasrc,experimentdetails_file))
trials_df = pd.read_csv(os.path.join(datasrc,alltrials_file))

with open(montagefile) as f: #open the montage file provided by Debbie
    montage = json.loads(f.read())
#change channel names to match standard naming convention in mne standard montage
montage['ch-1']='Fp1'
montage['ch-2']='Fpz'
montage['ch-3']='Fp2'

Fetch the data and basic summarization

1. Load the block
2. Gather the summary of the EEG channels
3. Gather summary of the triggers
4. Create the event array in mne format
5. Load EEG into mne

In [8]:
#change the eeg channel names in xdf to standard channel names using the montage file
def get_channel_names(temp,n):
    ch_names = []
    for i in range(n):
        n = temp[i]['label'][0].replace("ExG","ch-")
        ch_names.append(montage[n] if "ch-" in n else n) #only eeg channels are mapped, the aux channels are not
    return ch_names

#save files
def save_file(f,ftype,subj,block):
    save_dir = os.path.join(datasrc,subj_dir,subj,dir_structure[ftype])
    
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    if ftype in ['events','filtered','cleaned']:
        fend = '_eeg.fif'
    else:
        fend = '_epo.fif'
        
    fname = f"block_{str(block)}{fend}"
    f.save(os.path.join(save_dir,fname),overwrite=True)
    
    #when saving the epoched mne file also save a numpy version 
    if ftype == 'epoched':
        epoch_array = f.get_data() #convert mne to numpy array
        fname = f"block_{str(block)}_epo.npy"
        with open(os.path.join(save_dir,fname), 'wb') as f:
            np.save(f, epoch_array)
            
    
    
            
    

In [21]:
def get_eeg_and_preprocess(df):
    out = 'ok'
    subj = df['subj_id'].item()
    blk = df['block'].item()
    event_streams = literal_eval(df['trig_streams'].item())
    eeg_stream = df['eeg_stream'].item()
    stim_stream = df['stim_stream'].item()
    if (eeg_stream == 99) |  (stim_stream == 99) | ('99' in event_streams):
        print(" All datastreams not found. Skipped")
        out = 'skipped'
    else:
        try:
            print("start...")
            data, header = pyxdf.load_xdf(df['filename'].item()) #read the xdf
            eeg_time = data[eeg_stream]["time_stamps"] - data[eeg_stream]["time_stamps"][0] #get eeg time points 
            sr = data[eeg_stream]['info']['effective_srate'] #sampling rate of eeg
            n_ch = int(data[eeg_stream]['info']['channel_count'][0]) #no. of eeg channels
            ch_typ= ['eeg'] * n_ch #set channel type for mne
            ch_names = get_channel_names(
                data[eeg_stream]['info']['desc'][0]['channels'][0]['channel'],
                n_ch)
            
            #map the trial start cue i.e. the first index in the event_streams to ecent array for mne
            tdf = trials_df.loc[(trials_df['subj_id']== subj) &
                                (trials_df['block']==blk),
                               'condition']
            tdf = tdf.map(trial_event_map)
            x = np.searchsorted(data[eeg_stream]["time_stamps"],
                                data[int(event_streams[0])]["time_stamps"])
            events = np.zeros((x.shape[0],3),dtype=int)
            events[:,0] = x
            events[:,2] = tdf.to_numpy(dtype=int)
            
            #create mne info
            info = mne.create_info(ch_names, ch_types=ch_typ, sfreq=sr)
            
            #mne data array should be in (nChannel,nSamples) whereas xdf stores in (nSamples,nChannel)
            eeg = mne.io.RawArray(np.transpose(data[eeg_stream]["time_series"]), info) 
            #drop auxiliaries and not needed channels
            eeg.drop_channels(['BIP65', 'BIP66', 'BIP67', 'BIP68', 'AUX69', 'AUX70', 'AUX71', 'AUX72'])
            #set the montage
            eeg.set_montage('standard_1020')
            #resample to 512Hz
            #Note: the raw and the event are resampled simultaneously so that they stay more or less in synch.
            eeg_resamp, events_resamp = eeg.resample(sfreq=512,events=events)
            
            del eeg
            gc.collect()
            
            #EEG is recorded with average reference. Re-refer to Cz
            eeg_resamp.set_eeg_reference(ref_channels=['Cz'])
            #save to events folder
            save_file(eeg_resamp,'events',subj,blk)
            
            #Filter resampled EEG
            #High pass 0.1 Hz
            #Low pass 120 Hz
            #notch 50Hz,100Hz
            eeg_resamp.filter(l_freq=0.1, h_freq=120)
            eeg_resamp.notch_filter(freqs=[50,100])
            #save to filtered folder
            save_file(eeg_resamp,'filtered',subj,blk)
            
            #epochs paramaters
            tmin = -0.1 # 100 msec before the event boundary
            tmax = 6.0 # each trial is 2.0 + 0.5 + 3.0 + 0.5...NOTE: first 0.5 sec of action is included
            baseline = (tmin,0) #i.e. the entire zone from tmin to 0
            epochs = mne.Epochs(eeg_resamp, events_resamp, event_id=trial_event_map, tmin=tmin, tmax=tmax)
            epochs.drop_bad()
            #save to rawepochs folder
            save_file(epochs,'rawepochs',subj,blk)
            
            #del epochs
            #gc.collect()
            
            #run asr to clean the filtered raw data
            # Apply the ASR
            asr = ASR(sfreq=eeg_resamp.info["sfreq"], cutoff=15)
            artifactfree = eeg_resamp.copy().crop(tmin=1*60, tmax=5*60)
            print(artifactfree.n_times)
            asr.fit(artifactfree)
            print('ASR fitted\n')
            for i in range(len(epochs)):
                epochs[i] = asr.transform(epochs[i])
            #save to cleaned folder
            #save_file(eeg_resamp,'cleaned',subj,blk)

            #epoch the cleaned eeg, use the parameter already set-up
            #clean_epochs = mne.Epochs(eeg_resamp, events_resamp, event_id=trial_event_map, tmin=tmin, tmax=tmax)
            #clean_epochs.drop_bad()
            #save to epoched folder
            save_file(epochs,'epoched',subj,blk)
            
            del epochs,eeg_resamp
            gc.collect()
            
        except Exception as e:
            print(e)
            out = 'failed'
    
    if out == 'ok':
        print("Success")
    else:
        print(out)
    
    return out

    

In [22]:
subjects = expt_df['subj_id'].unique()
results = []
for s in subjects:
    subj_df = expt_df[expt_df['subj_id']==s]
    for block in subj_df['block'].unique():
        print("\nProcessing Subject {} Block {} ##############".format(s,block))
        flag = get_eeg_and_preprocess(subj_df[subj_df['block']==block])
        results.append(f"Subject: {s} Block: {str(block)} Report: {flag}")
        print("...end")
        break
    break
    
print("FINISHED")         


Processing Subject P001 Block 1 ##############
start...
Creating RawArray with float64 data, n_channels=72, n_times=741888
    Range : 0 ... 741887 =      0.000 ...   724.499 secs
Ready.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Writing /net/store/nbp/projects/GTI_decoding/data/pilot/01_subjects_data/P001/eeg/preprocessed/00_events_added/block_1_eeg.fif
Closing /net/store/nbp/projects/GTI_decoding/data/pilot/01_subjects_data/P001/eeg/preprocessed/00_events_added/block_1_eeg.fif
[done]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz

/net/home/student/s/sbiswas/mambaforge-pypy3/envs/mne/lib/python3.8/site-packages/asrpy/asr_utils.py:121: RuntimeWarning: divide by zero encountered in true_divide
  cols = nbins / newX[mcurr]
/net/home/student/s/sbiswas/mambaforge-pypy3/envs/mne/lib/python3.8/site-packages/asrpy/asr_utils.py:122: RuntimeWarning: invalid value encountered in multiply
  H = newX[:m] * cols
/net/home/student/s/sbiswas/mambaforge-pypy3/envs/mne/lib/python3.8/site-packages/asrpy/asr.py:738: RuntimeWarning: invalid value encountered in true_divide
  wz[ichan] = (Y - mu) / sig


ASR fitted

Loading data for 1 events and 3124 original time points ...
all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)
failed
...end
FINISHED


In [35]:
for r in results:
    print(r)

Subject: P001 Block: 1 Report: ok
